# Dask Bag

Материалы: 
* Макрушин С.В. Лекция 12: Map-Reduce
* https://docs.dask.org/en/latest/bag.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Подсчитайте длину (в кол-ве символов) каждого предложения.

In [1]:
import re
import dask.bag as db
import json

In [2]:
f = open('Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt')
s = f.read()
sentences = re.split(r'(?<=[^А-Я].[.?]) +(?=[А-Я])', s)
lens = [len(sentence) for sentence in sentences]
lens

[427,
 41,
 60,
 90,
 48,
 69,
 140,
 130,
 100,
 36,
 32,
 74,
 50,
 52,
 56,
 96,
 30,
 79,
 147,
 190,
 169,
 103,
 104,
 508,
 334,
 43,
 160,
 89,
 300,
 272,
 178,
 37,
 182,
 66,
 435,
 143,
 32,
 58,
 108,
 92,
 41,
 213,
 43,
 56,
 318,
 79,
 146,
 48,
 176,
 181,
 200,
 327,
 183,
 262,
 139,
 153,
 111,
 81,
 121,
 142,
 56,
 104,
 718,
 44,
 88,
 33,
 316,
 88,
 131,
 95,
 194,
 354,
 172,
 70,
 85,
 129,
 90,
 35,
 27,
 92,
 97,
 78,
 424,
 77,
 246,
 526,
 104,
 64,
 101,
 60,
 447,
 46,
 45,
 50,
 49,
 575,
 439,
 143,
 105,
 68,
 28,
 260,
 327,
 261,
 71,
 61,
 112,
 195,
 129,
 52,
 115,
 28,
 234,
 187,
 11,
 45,
 318,
 147,
 123,
 135,
 104,
 65,
 192,
 89,
 47,
 326,
 87,
 122,
 288,
 98,
 24,
 79,
 203,
 22,
 170,
 89,
 41,
 127,
 93,
 292,
 111,
 271,
 54,
 26,
 184,
 51,
 24,
 182,
 84,
 35,
 205,
 144,
 91,
 385,
 264,
 189,
 86,
 257,
 116,
 175,
 84,
 223,
 43,
 121,
 40,
 136,
 111,
 83,
 85,
 103,
 301,
 161,
 194,
 83,
 216,
 119,
 51,
 154,
 66,
 90,
 99,

2. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Выведите предложения, длина которых не более 10 символов.

In [3]:
for sentence in sentences:
    if len(sentence) <= 10:
        print(sentence)

Как зачем?
Прощайте.
Понимаю-с.
Так ли?
Не знаешь?
Ведь так?
Полина?


3. На основе списка предложений из задачи 1-2 создайте `dask.bag`. Рассчитайте среднюю длину предложений в тексте.

In [11]:
new_bag = db.from_sequence(lens)
mean_len = new_bag.mean()
f"Средняя длина одного предложения в тексте - {float(mean_len):.2f} символов"

'Средняя длина одного предложения в тексте - 148.87 символов'

4. На основе файла `addres_book.json` создайте `dask.bag`. Посчитайте количество мобильных и рабочих телефонов в наборе данных

## Лабораторная работа 12

In [2]:
import dask.bag as db
import json
import re
import pandas as pd

1. В файлах архиве `reviews_full.zip` находятся файлы, содержащие информацию об отзывах к рецептам в формате JSON Lines. Отзывы разделены на файлы в зависимости от оценки (например, в файле `reviews_1.json` находятся отзывы с оценкой 1). Считайте файлы из этого архива в виде `dask.bag`. Преобразуйте текстовое содержимое файлов в объекты python (с помощью модуля `json`). Выведите на экран первые 5 элементов полученного `bag`.

In [3]:
bag = db.read_text('reviews_*.json').map(json.loads)

In [4]:
bag.take(5)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.'},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': '2006-06-14',
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten'},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': '1985-11-19',
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will be making this often.'},
 {'u

2. Модифицируйте функцию разбора JSON таким образом, чтобы в каждый словарь c информацией об отзыве добавить ключ `rating`. Значение получите на основе названия файла (см. аргумент `include_path`), использовав для этого регулярное выражение.

In [5]:
def parse_json(data):
    ddata = json.loads(data[0])
    ddata['rating'] = int(re.findall(r'reviews_(\d).+$', data[1])[0])
    return ddata

In [6]:
bag = db.read_text('reviews_*.json', include_path=True).map(parse_json)
bag.take(3)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.',
  'rating': 0},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': '2006-06-14',
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten',
  'rating': 0},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': '1985-11-19',
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will

3. Посчитайте количество отзывов в исходном датасете.

In [7]:
bag.pluck('review').count().compute()

9057540

4. Отфильтруйте `bag`, сохранив только отзывы, оставленные в 2014 и 2015 годах.

In [8]:
bg = bag.filter(lambda x: x['date'][:4] in ('2014', '2015'))

In [9]:
bg.take(7)

({'user_id': 229850,
  'recipe_id': 1300038,
  'date': '2014-10-03',
  'review': 'Took this to a New Year&#039;s Eve Party. Everyone loved it! It&#039;s absolutely perfect, the flavor, the crunch, just delicious!',
  'rating': 0},
 {'user_id': 2706705,
  'recipe_id': 133747,
  'date': '2015-05-08',
  'review': 'Simple and easy way to enjoy a slice of pizza any time!  Well-toasted bread is the key - really toast it!  I put a bit of pizza sauce underneath my cheese for a more pizza-like flavor.  I used sourdough bread & medium cheddar cheese.  Fast & fun!  Great idea!  Made for 1-2-3 Hits Tag Game.',
  'rating': 0},
 {'user_id': 945545,
  'recipe_id': 898468,
  'date': '2015-06-30',
  'review': 'Delish!  I wanted to make this spicy so I used hot enchilada sauce and jalapeno refried beans.  I forgot to buy the onions so I doctored up the beans with onion powder and granulated garlic.  Added the olives under the cheese and baked, uncovered, for the 25 minutes.  Served with pico de gallo, s

5. Выполните препроцессинг отзывов:
    * привести строки к нижнему регистру
    * обрезать пробельные символы в начале и конце строки
    * удалите все символы, кроме английских букв и пробелов
    
Примените препроцессинг ко всем записям из `bag`, полученного в задании 4.

In [10]:
def preproc():
    rew['review'] = re.sub(r'[^a-z ]+','', rew['review'].lower().strip())
    return rew

In [11]:
bg5 = bg.map(preproc)

In [12]:
bg5.take(3)

({'user_id': 229850,
  'recipe_id': 1300038,
  'date': '2014-10-03',
  'review': 'took this to a new years eve party everyone loved it its absolutely perfect the flavor the crunch just delicious',
  'rating': 0},
 {'user_id': 2706705,
  'recipe_id': 133747,
  'date': '2015-05-08',
  'review': 'simple and easy way to enjoy a slice of pizza any time  welltoasted bread is the key  really toast it  i put a bit of pizza sauce underneath my cheese for a more pizzalike flavor  i used sourdough bread  medium cheddar cheese  fast  fun  great idea  made for  hits tag game',
  'rating': 0},
 {'user_id': 945545,
  'recipe_id': 898468,
  'date': '2015-06-30',
  'review': 'delish  i wanted to make this spicy so i used hot enchilada sauce and jalapeno refried beans  i forgot to buy the onions so i doctored up the beans with onion powder and granulated garlic  added the olives under the cheese and baked uncovered for the  minutes  served with pico de gallo sour cream and avocado chunks  fantastic  tha

6. Посчитайте количество отзывов в датасете, полученном в результате решения задачи 5. В случае ошибок прокомментируйте результат и исправьте функцию препроцессинга.

In [13]:
bg5.pluck('review').count().compute()

735274

7. Посчитайте, как часто в наборе, полученном в задании 5, встречается та или иная оценка

In [14]:
bg5.pluck('rating').frequencies().compute()

[(0, 42472), (1, 9246), (2, 9380), (3, 26532), (4, 119413), (5, 528231)]

8. Найдите среднее значение `rating` в выборке

In [15]:
bg5.pluck('rating').mean().compute()

4.388036296673077

9. Используя метод `foldby`, подсчитать максимальную длину отзывов в зависимости от оценки `rating` в наборе, полученном в задании 5.

In [16]:
def binop(total, value):
    if total > len(value['review']):
        return total
    return len(value['review'])

def combine(left, right):
    if left > right:
        return left
    return right

In [17]:
bg5.foldby('rating', binop, 0, combine, 0).compute()

[(0, 6548), (1, 2868), (2, 2834), (3, 3174), (4, 6548), (5, 5343)]